In [37]:
# =========================
# CELL 1: Setup + Load + Prepare
# Mục đích:
# 1) Nạp thư viện cần dùng (xử lý dữ liệu, baseline ML, DL PyTorch)
# 2) Đọc data.csv vào df
# 3) Chuẩn hoá nhãn theo phương pháp nhị phân: 0 -> 0, (1/2) -> 1 để thành bài toán nhị phân (0 là không bị tiểu đương, 1/2: có nguy cơ/bị tiểu đường)
# 4) Tách X (21 feature) và y (label) để chuẩn bị train model
# 5) In thống kê để kiểm tra dữ liệu đã đúng và thấy mức lệch lớp
# =========================

import os, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
import joblib

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Đọc dữ liệu
df = pd.read_csv("data.csv")

# Nhãn nhị phân (binary)
y = (df["Diabetes_binary"] > 0).astype(np.int64).values     # 0->0, 1/2->1
X = df.drop(columns=["Diabetes_binary"]).astype(np.float32).values  # 21 feature

# Kiểm tra nhanh
print("X shape:", X.shape)  # (số mẫu, số feature)
print("y distribution:", dict(zip(*np.unique(y, return_counts=True))))  # xem lệch lớp


X shape: (269131, 21)
y distribution: {np.int64(0): np.int64(194377), np.int64(1): np.int64(74754)}


In [38]:
# =========================
# CELL 2: Split + Scale
# Mục đích:
# 1) Chia dữ liệu thành train/val/test để đánh giá công bằng (test không được nhìn lúc train)
# 2) Dùng stratify để giữ đúng tỉ lệ lớp ở mọi tập (quan trọng vì dữ liệu lệch lớp)
# 3) Chuẩn hoá feature bằng StandardScaler để model học ổn định (các cột về cùng thang đo)
# 4) Lưu scaler để sau này inference/reproduce kết quả giống y chang
# =========================

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Tách thêm validation từ train
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Scale: fit trên train, rồi transform val/test
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train).astype(np.float32)
X_val_s   = scaler.transform(X_val).astype(np.float32)
X_test_s  = scaler.transform(X_test).astype(np.float32)

# Lưu scaler để tái sử dụng
joblib.dump(scaler, "scaler.joblib")


['scaler.joblib']

In [39]:
# =========================
# CELL 3: Baseline Logistic Regression
# Mục đích:
# 1) Train một mô hình đơn giản (Logistic Regression) làm "baseline"
# 2) Baseline là mốc để chứng minh mô hình Deep Learning có cải thiện thật sự
# 3) Đánh giá baseline bằng ROC-AUC + confusion matrix + classification report
# 4) Lưu baseline model để nộp và reproduce
# =========================

lr = LogisticRegression(max_iter=2000, n_jobs=-1)
lr.fit(X_train_s, y_train)

# Dự đoán xác suất và nhãn
proba_lr = lr.predict_proba(X_test_s)[:, 1]
pred_lr  = (proba_lr >= 0.5).astype(int)

# Đánh giá baseline
print("=== Logistic Regression (Baseline) ===")
print("ROC-AUC:", roc_auc_score(y_test, proba_lr))
print(confusion_matrix(y_test, pred_lr))
print(classification_report(y_test, pred_lr, digits=4))

# Lưu baseline
joblib.dump(lr, "logreg_baseline.joblib")


=== Logistic Regression (Baseline) ===
ROC-AUC: 0.8046563272103694
[[35307  3569]
 [ 8972  5979]]
              precision    recall  f1-score   support

           0     0.7974    0.9082    0.8492     38876
           1     0.6262    0.3999    0.4881     14951

    accuracy                         0.7670     53827
   macro avg     0.7118    0.6541    0.6686     53827
weighted avg     0.7498    0.7670    0.7489     53827



['logreg_baseline.joblib']

In [40]:
# =========================
# CELL 4: MLP Train (Deep Learning) + Optimization
# Mục đích:
# 1) Chuẩn bị DataLoader để train theo mini-batch (train nhanh và ổn định)
# 2) Định nghĩa mô hình MLP (feedforward neural network) đúng yêu cầu đề bài
# 3) Train model và áp dụng ít nhất 2 optimization techniques:
#    - Optimization #1: pos_weight (xử lý lệch lớp)
#    - Optimization #2: Dropout + Early stopping (giảm overfit, dừng đúng lúc)
# 4) Lưu model train xong để nộp và tái chạy
# =========================

# --- DataLoader ---
train_ds = TensorDataset(torch.tensor(X_train_s), torch.tensor(y_train).float())
val_ds   = TensorDataset(torch.tensor(X_val_s),   torch.tensor(y_val).float())
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=1024, shuffle=False)

# --- MLP model ---
class MLP(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 64), nn.ReLU(),
            nn.Dropout(0.3),               # (Optimization) giảm overfit
            nn.Linear(64, 32), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1)               # output 1 logit (binary)
        )
    def forward(self, x):
        return self.net(x).squeeze(1)

model = MLP(X_train_s.shape[1])

# --- Optimization #1: pos_weight để xử lý lệch lớp ---
pos = y_train.sum()
neg = len(y_train) - pos
pos_weight = torch.tensor([neg / pos], dtype=torch.float32)

# Loss + Optimizer (weight_decay = L2 regularization)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# --- Optimization #2: Early stopping theo val AUC ---
best_auc, patience, bad = -1.0, 5, 0
best_state = None

for epoch in range(30):
    # Train
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

    # Validate AUC
    model.eval()
    vp, vy = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            prob = torch.sigmoid(model(xb)).cpu().numpy()
            vp.append(prob); vy.append(yb.cpu().numpy())

    vp = np.concatenate(vp); vy = np.concatenate(vy)
    auc = roc_auc_score(vy, vp)

    print(f"epoch {epoch+1:02d} | val_auc={auc:.4f}")

    # Early stopping: lưu best model
    if auc > best_auc + 1e-4:
        best_auc = auc
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping.")
            break

# Load model tốt nhất và lưu file model
model.load_state_dict(best_state)
torch.save(model.state_dict(), "mlp_model.pt")
print("Saved: mlp_model.pt")


epoch 01 | val_auc=0.8023
epoch 02 | val_auc=0.8049
epoch 03 | val_auc=0.8057
epoch 04 | val_auc=0.8063
epoch 05 | val_auc=0.8066
epoch 06 | val_auc=0.8066
epoch 07 | val_auc=0.8069
epoch 08 | val_auc=0.8069
epoch 09 | val_auc=0.8070
epoch 10 | val_auc=0.8071
epoch 11 | val_auc=0.8072
epoch 12 | val_auc=0.8071
epoch 13 | val_auc=0.8073
epoch 14 | val_auc=0.8072
epoch 15 | val_auc=0.8073
epoch 16 | val_auc=0.8073
epoch 17 | val_auc=0.8074
epoch 18 | val_auc=0.8076
epoch 19 | val_auc=0.8077
epoch 20 | val_auc=0.8076
epoch 21 | val_auc=0.8077
epoch 22 | val_auc=0.8078
epoch 23 | val_auc=0.8077
epoch 24 | val_auc=0.8076
epoch 25 | val_auc=0.8078
epoch 26 | val_auc=0.8076
Early stopping.
Saved: mlp_model.pt


In [41]:
# =========================
# CELL 5: Evaluate + Inference
# Mục đích:
# 1) Đánh giá mô hình MLP trên test set (phần dữ liệu chưa thấy khi train)
# 2) In ROC-AUC + confusion matrix + report để so sánh với baseline
# 3) Demo inference: nhập 1 mẫu mới -> scale -> model trả xác suất nguy cơ
# =========================

# --- Test evaluation ---
model.eval()
with torch.no_grad():
    prob_mlp = torch.sigmoid(model(torch.tensor(X_test_s))).cpu().numpy()

pred_mlp = (prob_mlp >= 0.5).astype(int)

print("=== MLP (Optimized) ===")
print("ROC-AUC:", roc_auc_score(y_test, prob_mlp))
print(confusion_matrix(y_test, pred_mlp))
print(classification_report(y_test, pred_mlp, digits=4))

# --- Inference demo (1 sample) ---
x_new = X_test[:1].astype(np.float32)                 # giả lập 1 người mới
x_new_s = scaler.transform(x_new).astype(np.float32)  # scale giống lúc train

with torch.no_grad():
    p = torch.sigmoid(model(torch.tensor(x_new_s))).item()

print("Predicted diabetes risk probability:", p)
print("Predicted class:", int(p >= 0.5))


=== MLP (Optimized) ===
ROC-AUC: 0.8137359530242804
[[26669 12207]
 [ 3173 11778]]
              precision    recall  f1-score   support

           0     0.8937    0.6860    0.7762     38876
           1     0.4911    0.7878    0.6050     14951

    accuracy                         0.7143     53827
   macro avg     0.6924    0.7369    0.6906     53827
weighted avg     0.7818    0.7143    0.7286     53827

Predicted diabetes risk probability: 0.1568731963634491
Predicted class: 0
